In [12]:
import pandas as pd
import numpy as np
from datetime import timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns=None
pd.options.mode.chained_assignment = None

In [2]:

triggers_actions = pd.read_csv(r'C:\Users\User\triggers\triggers_actions_10_2f.csv',index_col=0)


In [3]:
triggers_actions['date'] = pd.to_datetime(triggers_actions['date'], errors='coerce')
triggers_actions['action_date'] = pd.to_datetime(triggers_actions['action_date'], errors='coerce')
triggers_actions.info()
triggers_actions.head(10)


<class 'pandas.core.frame.DataFrame'>
Index: 8614925 entries, 1973177 to 77
Data columns (total 9 columns):
 #   Column                                 Dtype         
---  ------                                 -----         
 0   guid                                   object        
 1   date                                   datetime64[ns]
 2   trigger                                int64         
 3   type                                   int64         
 4   action_date                            datetime64[ns]
 5   result                                 float64       
 6   last_action_date                       object        
 7   time_since_last_action                 float64       
 8   triggers_since_last_action_or_14_days  int64         
dtypes: datetime64[ns](2), float64(2), int64(3), object(2)
memory usage: 657.3+ MB


,guid,date,trigger,type,action_date,result,last_action_date,time_since_last_action,triggers_since_last_action_or_14_days
1973177,0187808e-fdc5-515b-59d2-7735eef5f2a1,2024-05-30 12:30:35,178,1,NaT,NaN,NaN,NaN,0
5014291,0187808f-5d64-f73b-f106-1e8dabdb2b4e,2024-05-28 09:15:31,126,1,NaT,NaN,NaN,NaN,0
5013911,0187808f-5d64-f73b-f106-1e8dabdb2b4e,2024-05-28 09:15:32,126,1,NaT,NaN,NaN,NaN,1
3718116,01878092-1df5-08c2-9419-1865b6a92257,2024-05-29 07:19:03,159,1,NaT,NaN,NaN,NaN,0
4670329,01878092-3dec-4d42-fad6-22f56b7a0497,2024-05-28 12:21:57,2056,2,NaT,NaN,NaN,NaN,0
8076664,01878094-10d3-c9c8-5289-7a7608105b54,2024-05-26 07:15:30,2053,2,NaT,NaN,NaN,NaN,0
8071143,01878094-10d3-c9c8-5289-7a7608105b54,2024-05-26 07:15:39,2053,2,NaT,NaN,NaN,NaN,1
7325160,01878094-8416-741e-b5a5-fef85a5cf0df,2024-05-26 16:00:23,6,1,NaT,NaN,NaN,NaN,0
7324890,01878094-8416-741e-b5a5-fef85a5cf0df,2024-05-26 16:00:24,6,1,NaT,NaN,NaN,NaN,1
7223556,01878094-8416-741e-b5a5-fef85a5cf0df,2024-05-26 17:35:32,6,1,NaT,NaN,NaN,NaN,2


In [4]:
# 1. Находим предыдущую дату действия для каждого пользователя
triggers_actions['previous_action_date'] = triggers_actions.groupby('guid')['action_date'].shift()


In [5]:
max_value= triggers_actions['time_since_last_action'].max()

In [6]:
max_value

6.242893518518518

In [7]:

# 2. Вычисляем время от предыдущего действия
# Если предыдущее действие есть, считаем разницу; если его нет, ставим 14 дней
triggers_actions['time_since_last_action'] = triggers_actions.apply(
    lambda row: (row['date'] - row['previous_action_date']).total_seconds() / (3600 * 24)
    if pd.notna(row['action_date']) and pd.notna(row['previous_action_date'])
    else (max_value + 14 if pd.isna(row['previous_action_date']) and pd.notna(row['action_date']) else row['time_since_last_action']),
    axis=1
)


In [8]:

# 3. Удаляем вспомогательную колонку, если она больше не нужна
triggers_actions = triggers_actions.drop(columns=['previous_action_date','last_action_date'])

In [14]:
triggers_actions.to_csv(r'C:\Users\User\triggers\triggers_actions_10_2f.csv',index=False)

In [ ]:
triggers_actions.head(300)

In [ ]:
duplicate_analysis = triggers_duplicates.groupby(['guid', 'trigger']).size().reset_index(name='count')
duplicate_analysis.head()
duplicate_analysis.sort_values(by='count', ascending=False).head(10)

In [ ]:

def time_since_last_action(group):
    date_min = group['date'].min()

    def calculate_row(row):
        nonlocal date_min
        if pd.notna(row['last_action_date']):
            row['time_since_last_action'] = (row['date'] - row['last_action_date']).total_seconds() / (3600 * 24)
        else:
            row['time_since_last_action'] = (row['date'] - date_min).total_seconds() / (3600 * 24)
            
        return row

    return group.apply(calculate_row,axis=1)
triggers_actions = triggers_actions.groupby('guid',group_keys=False).apply(time_since_last_action)

In [ ]:

# проходим по каждой группе guid
for guid, group in triggers_actions.groupby('guid'):
    # сохраняем дату последнего взаимодействия и создаем переменную для подсчета триггеров
    last_action_date = pd.NaT
    trigger_count = 0
    
    # проходим по строкам группы
    for i, row in group.iterrows():
        current_date = row['date']
        
        # если было взаимодействие, обновляем дату последнего действия
        if pd.notna(row['action_date']):
            last_action_date = row['action_date']
        
        # если взаимодействия не было, считаем за последние 14 дней или с момента последнего взаимодействия
        if pd.isna(last_action_date):
            start_date = current_date - pd.Timedelta(days=14)
        else:
            start_date = last_action_date
        
        # фильтруем триггеры в нужный диапазон времени
        trigger_count = group[(group['date'] >= start_date) & (group['date'] < current_date)].shape[0]
        
        # добавляем результат в список
        triggers_since_last_action_or_14_days.append(trigger_count)

triggers_actions['triggers_since_last_action_or_14_days'] = triggers_since_last_action_or_14_days        

In [ ]:
# количество дней с последнего взаимодействия(time_since_last_action), заменим пропуски на максимальное значение в этой колонке плюс 10, чтобы показать 
# что взаимодействия не было 


max_value = triggers_actions['time_since_last_action'].max()

# заменяем NaN на max_value + 30, чтобы отразить, что взаимодействие было "очень давно"
triggers_actions['time_since_last_action'].fillna(max_value + 30, inplace=True)

### отдельно q_tables

In [ ]:
# Подготовка данных для triggers_category
filtered_data_triggers = filtered_data[['guid','result', 'triggers_category']].dropna()

# уникальные значения для triggers_category
triggers_categories = filtered_data_triggers['triggers_category'].unique()

# инициализация Q-таблицы
q_table_triggers = pd.DataFrame(index=triggers_categories, columns=[0, 1])
q_table_triggers.fillna(0, inplace=True)


alpha = 0.1  # скорость обучения
gamma = 0.9  # коэффициент дисконтирования
epsilon = 0.1  # вероятность выбора случайного действия 

# количество эпизодов для обучения

num_episodes = 1000

for episode in range(num_episodes):
    # случайный выбор состояния (triggers_category)
    state = random.choice(triggers_categories)
    
    # выбор действия (0 или 1) с использованием ε-жадной стратегии
    if random.uniform(0, 1) < epsilon:
        action = random.choice([0, 1])  # случайное действие
    else:
        action = q_table_triggers.loc[state].idxmax()  # действие с наибольшим Q-значением

    # получение награды из filtered_data_triggers
    reward = filtered_data_triggers[(filtered_data_triggers['triggers_category'] == state) & (filtered_data_triggers['result'] == action)]['result'].sum()

    # обновление Q-таблицы
    old_value = q_table_triggers.loc[state, action]
    next_max = q_table_triggers.loc[state].max()  # максимальное Q-значение для следующего состояния
    new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
    q_table_triggers.loc[state, action] = new_value

# вывод Q-таблицы после обучения
print("Q-таблица для triggers_category:")
q_table_triggers

In [ ]:

# оставляем только необходимые столбцы
filtered_data_reduced = filtered_data[['guid', 'time_category', 'result']].copy()
# уникальные значения для time_category
time_categories = filtered_data_reduced['time_category'].unique()

time_categories


# гиперпараметры
alpha = 0.1  # скорость обучения
gamma = 0.9  # коэффициент дисконтирования
epsilon = 0.1  # вероятность выбора случайного действия

# уникальные значения для time_category
time_categories = filtered_data_reduced['time_category'].unique()

# инициализация Q-таблицы
q_table_time = pd.DataFrame(index=time_categories, columns=[0, 1])
q_table_time.fillna(0, inplace=True)

# количество эпизодов для обучения
num_episodes = 1000

for episode in range(num_episodes):
    # случайный выбор состояния (time_category)
    state = random.choice(time_categories)
    
    # выбор действия (0 или 1) с использованием ε-жадной стратегии
    if random.uniform(0, 1) < epsilon:
        action = random.choice([0, 1])  # случайное действие
    else:
        action = q_table_time.loc[state].idxmax()  # действие с наибольшим Q-значением

    # получение награды из filtered_data
    reward = filtered_data_reduced[(filtered_data_reduced['time_category'] == state) & (filtered_data_reduced['result'] == action)]['result'].sum()

    # обновление Q-таблицы
    old_value = q_table_time.loc[state, action]
    next_max = q_table_time.loc[state].max()  # максимальное Q-значение для следующего состояния
    new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
    q_table_time.loc[state, action] = new_value

# вывод Q-таблицы после обучения
print("Q-таблица для time_category:")
q_table_time

In [ ]:
Обученная Q-таблица для триггеров:
                              0            1
(-0.001, 103.448]      0.008120  1823.879598
(103.448, 206.897]     2.040484   234.170422
(206.897, 310.345]     4.575213   136.982657
(310.345, 413.793]     0.328509   169.665299
(413.793, 517.241]     4.039899   127.636201
(517.241, 620.69]      0.139006   334.630424
(620.69, 724.138]      0.619936   310.299985
(724.138, 827.586]    12.422875   206.108228
(827.586, 931.034]    51.478178   192.593398
(931.034, 1034.483]   14.283345   188.108576
(1034.483, 1137.931]  13.255149    90.497976
(1137.931, 1241.379]   2.447203   108.709815
(1241.379, 1344.828]  16.093268   102.852841
(1344.828, 1448.276]   7.112704   154.300417
(1448.276, 1551.724]  21.075338   130.662544
(1551.724, 1655.172]   4.633036    75.889918
(1655.172, 1758.621]   3.456295    78.761337
(1758.621, 1862.069]  19.326149   175.466763
(1862.069, 1965.517]   0.037779   124.223095
(1965.517, 2068.966]  26.179429   146.735494
(2068.966, 2172.414]   6.812260   107.525947
(2172.414, 2275.862]   0.670578    99.730574
(2275.862, 2379.31]    5.578079    72.666933
(2379.31, 2482.759]   12.865724   113.368923
(2482.759, 2586.207]   0.063691    64.012850
(2586.207, 2689.655]  38.331977   238.057277
(2689.655, 2793.103]   1.916197    89.399930
(2793.103, 2896.552]  17.381137   123.664824
(2896.552, 3000.0]    13.889713   251.999847
(3000.0, 49683.0]      5.094997   273.217278


Обученная Q-таблица для времени:
                          0           1
NaN                0.582474  153.607047
(13.999, 14.56]   31.421358  126.621601
(16.801, 17.361]  59.016608  222.692051
(17.361, 17.922]   9.835492  101.416945
(17.922, 18.482]  17.865934  146.268476
(15.681, 16.241]   5.069464  302.318873
(14.56, 15.12]     9.154061  157.604996
(19.042, 19.602]   2.534369   98.253746
(19.602, 20.163]   5.164727  142.090995
(20.723, 21.283]   0.000000  108.921834
(15.12, 15.681]    1.401793   87.762255
(20.163, 20.723]   0.000000  152.976301
(21.283, 21.843]  29.725389  109.477113
(21.843, 22.403]  15.011619  252.229577
(22.403, 22.964]  24.640669  107.861820
(22.964, 23.524]  16.113729  110.338672
(23.524, 24.084]   2.269416  115.678368
(24.084, 24.644]  20.326522  351.382934
(25.765, 26.325]   0.000000  229.237876
(26.325, 26.885]  38.580322  139.787956
(26.885, 27.445]  12.098625   87.869767
(16.241, 16.801]   3.702321  148.086658
(28.566, 29.126]   0.027000   98.742623
(25.205, 25.765]  21.845816  198.284020
(27.445, 28.006]  52.816418  127.351021
(18.482, 19.042]  10.343873  110.174389
(28.006, 28.566]  24.684920  380.756261
(24.644, 25.205]  32.402712   97.080195
(29.126, 29.686]  13.648345  104.022384
(29.686, 30.247]   0.000000  108.076693
(30.247, 30.807]  18.378030  160.176254


In [ ]:
# Время

# Получаем максимальное значение в столбце triggers_since_last_action
min_time = filtered_data['time_since_last_action'].min()
max_time = filtered_data['time_since_last_action'].max()

# Создаем интервалы
# Первая категория будет от 0 до 15,000, разбитая на 4 равные части
bins = np.linspace(),max_time, num=30).tolist()  # 4 категории от 0 до 15,000

# Добавляем одну категорию от 15,000 до max_trigger_value
bins.append(max_trigger_value)  # Добавляем верхнюю границу

# Убедимся, что значения в bins уникальны
bins = sorted(set(bins))

# Создаем категориальный столбец
filtered_data['triggers_category'] = pd.cut(filtered_data['triggers_since_last_action'], bins=bins, include_lowest=True)

# Проверяем результат
print(filtered_data['triggers_category'].value_counts())

In [ ]:

### Random forest with probabilities
X = filtered_data[['interaction_prob', 'time_since_last_action', 'triggers_since_last_action', 'last_successful_type']]
y = filtered_data['result']

# делим на тренировочные и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# создаем и обучаем случайный лес с взвешиванием классов
rf = RandomForestClassifier(class_weight='balanced')
rf.fit(X_train, y_train)

# делаем предсказания
y_pred = rf.predict(X_test)
y_pred_prob = rf.predict_proba(X_test)

# оценка модели
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_prob, multi_class='ovr'))
print("Classification Report:\n", classification_report(y_test, y_pred))
# важность признаков
importances = rf.feature_importances_

# создаем DataFrame для удобного отображения
feature_names = X_train.columns  # X_train — это датафрейм с признаками
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

# сортируем признаки по важности
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# визуализация важности признаков
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Важность признаков в Random Forest', fontsize=14)
plt.xlabel('Важность', fontsize=12)
plt.ylabel('Признаки', fontsize=12)
plt.show()